# 다층을 가진 신경망

In [1]:
import numpy as np
import csv
import pandas as pd

In [2]:
%run NN_Softmax.ipynb

In [3]:
# 신경망의 파라미터 생성 함수
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt

    pm_hidden = allocate_param_pair([input_cnt, hidden_cnt])
    pm_output = allocate_param_pair([hidden_cnt, output_cnt])

In [4]:
def allocate_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])

    return {'w':weight, 'b':bias}

In [5]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden

    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_hidden['w']) + pm_output['b']

    return output, [x,hidden]

def relu(x):
    return np.maximum(x,0)

In [6]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    x, hidden = aux

    g_output_w_out = hidden.transpose()
    G_w_output = np.matmul(g_output_w_out, G_output)
    G_b_output = np.sum(G_output, axis=0)

    # 출력 계층의 파라미터 갱신
    pm_output['w'] -= LEARNING_RATE * G_w_output
    pm_output['b'] -= LEARNING_RATE * G_b_output

    G_hidden = G_hidden * relu_derv(hidden)

    # 은닉 계층의 파라미터 갱신
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis=0)

    pm_hidden['w'] -= LEARNING_RATE * G_w_hid
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid

In [7]:
def relu_derv(y):
    return np.maximum(np.sign(y),0)

In [8]:
def set_hidden(info):
    '''
    은닉 계층의 수와 폭을 설정하여 값을 넘겨주는 역할을 한다.
    '''
    global hidden_cnt, hidden_config
    # 일반적인 하나의 은닉 계층이라면
    if isinstance(info,int):
        hidden_cnt = info
        hidden_config = None

    else:
        # 리스트값이라면 그대로 다층 퍼셉트론 수행
        hidden_config = info

In [9]:
def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}을 갖는 다층 퍼셉트론이 구축되었습니다.'.format(len(hidden_config)))
        init_model_hiddens()

    else:
        print("하나의 은닉계층을 갖는 다층 퍼셉트론이 구축되었습니다.")
        init_model_hidden1()

In [10]:
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)

    else:
        return forward_neuralnet_hidden1(x)

def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        return backprop_neuralnet_hiddens(G_output,hiddens)

    else:
        return backprop_neuralnet_hidden1(G_output, hiddens)

In [11]:
def init_model_hiddens():                   # 은닉 계층 초기화
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    pm_hiddens =[]                          # 은닉 계층이 저장될 빈 리스트 생성
    prev_cnt = input_cnt                    # input_cnt의 크기는 다음 게층의 입력 크기

    for hidden_cnt in hidden_config:        # hidden_config 리스트로 받아올 은닉 계층의 수와 폭을 반복문으로
        pm_hiddens.append(allocate_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt

    pm_output = allocate_param_pair([prev_cnt, output_cnt])

In [12]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux

    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out

    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)

        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['w'] -= LEARNING_RATE * G_b_hid

In [13]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens

    hidden = x
    hiddens = [x]

    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)

    output = np.matmul(hidden, pm_output['w']) + pm_output['b']

    return output, hiddens